In [1]:
from os.path import exists
from itertools import combinations
import csv
import time


In [10]:
# Import the data and do preprocessing

In [11]:
debugMain = False
#build lists for untouchedNodes()
listOfFacetConnections = []
listOfNodeConnections= []

#replace with whatever it needs to be

maxNodes = 48 #aka max depth?
maxNumOfConnections = 0
numNodes = 0
numFaces = 0
setOfFirst24Connections = set()
completeSetOfFaces = set()

# use this as main?
# grab shape file and import
flag = True
while flag:
    print("Filepath to import")
    filePath = input()
    if exists(filePath):
        with open(filePath, newline='') as file:
            reader = csv.reader(file)
            data = list(reader)
            for row in range(len(data)):
                listOfFacetConnections.append(set())
                for col in range(len(data[0])):
                    listOfFacetConnections[-1].add(int(data[row][col]))
                if len(listOfFacetConnections[-1]) > maxNumOfConnections:
                    maxNumOfConnections = len(listOfFacetConnections[-1])
            #print(listOfSets)
            if debugMain:
                print("maxNumOfConnections:", maxNumOfConnections)
            flag = False
            print("File Loaded")
            break
    else:
        print("Sorry, could not find that file")

#build additional preprocessing info
tempSet = set()
for i in listOfFacetConnections:
    tempSet.update(i)
numNodes = len(tempSet) #this shouldn't be different than len(listOfSets now that I think about it? unsure.)

numFaces = len(listOfFacetConnections)
for i in range(numNodes):
    listOfNodeConnections.append(set())
    
for i in range(len(listOfFacetConnections)):
    completeSetOfFaces.add(i+1) #offset by one because of our dataset
    
for face in range(len(listOfFacetConnections)):
    for node in listOfFacetConnections[face]:
        listOfNodeConnections[node-1].add(face+1)

if debugMain:
    print(len(completeSetOfFaces))
    print(completeSetOfFaces)
    print(len(listOfNodeConnections))
    print(listOfNodeConnections)

for i in range(24):
    setOfFirst24Connections.update(listOfNodeConnections[i])
    

Filepath to import
sixhundredcell_3_corrected.csv
File Loaded


In [ ]:
sixhundredcell_3_corrected.csv

In [ ]:
# Functions 

In [12]:
# This function takes an input set, a number of nodes to remove, and a number of nodes to put back, and tries all possible solutions
# in that given space. (If smart is false, but if smart is true, it only checks solutions that have nodes that connect to untouched 
# Facets, but still checks all of those)
def Backtrack(inputSet, numNodesToRemove, numReplacements = None, smart = True, override = False):
    # Basically, if not overriding this if statement, and your trying to do a smart, while removing 1 node,
    # Reason: I don't understand why, but the specific combo of smart and 1,1 leads to about 1/8th the outputs
    # as dumb output, and given that it takes less than a second on my bad PC, I think its a fine tradeoff in this specific situation
    if not override and smart and numNodesToRemove == 1 and numReplacements == 1:
        smart = False
    output = []
    if numReplacements == None:
        numReplacements = numNodesToRemove-1
    
    # cycle through all possible sets of nodes to remove
    for removedNodes in combinations(inputSet, numNodesToRemove):
        trialSet = inputSet.difference(removedNodes)
        
        #if smart, only pull replacements from nodes adj to missing faces
        if smart:        
            untouchedFaces = UntouchedFacets(trialSet)
            possibleReplacementNodes = FacetConnections(untouchedFaces)
        else:
            possibleReplacementNodes = fullSetOfNodes.difference(trialSet)
        
        # cycle through all possible replacements, (limited by numReplacements)
        # will be ran once with () as the subcombo if numreplacements == 0
        # This can be used for seeing if a single node (or set of nodes) can be dropped without losing any coverage
        for replacementNodes in combinations(possibleReplacementNodes, numReplacements): 
            if UntouchedFacets(trialSet.union(replacementNodes), setOutput = False) == 0:
                temp = trialSet.union(replacementNodes)
                flag = True
                for extantSolution in output:
                    if extantSolution.difference(temp) == set():
                        flag = False
                if flag:
                    output.append(temp)
                    #print("Solution:", output[-1])
                    
    return output


# Basically, this function will repeat the same Backtrack call on the results of the backtrack. This may be ideal to work down larger 
# solutions since doing even a 3-1 replacement takes a decent chunk of time, and any number greater than that starts to get crazy
# so by chaining 2 2-1 replacements, in effect, we can achieve a result similar to 3-1 in a much shorter timeframe (with wider solution 
# options aswell)
def chainBacktrack(inputSet, numNodesToRemove, numReplacements = None, smart = True, override = False):
    if(numNodesToRemove == numReplacements):
        print("Oh no! That'd cause an infinite loop. Try doing that but with Backtrack() instead")
        return
    
    results = Backtrack(inputSet, numNodesToRemove, numReplacements, smart, override)
    if results != []:
        temp = [] #new output
        for solution in results: 
            temp2 = chainBacktrack(solution, numNodesToRemove, numReplacements, smart, override)
            for i in temp2:
                temp.append(i)
                
        if temp != []:
            #print("hey", temp)
            return temp
    return results


# inputSet is a set of nodes, cheat auto adds the first 24 nodes, offset shifts all the values in inputSet down by one to work with the
# list properly, and setOutput changes the output from a number to a set of facets.
# if this returns 0, inputSet is a solution
# otherwise returns the number of faces not covered, OR if setOutput is true, the facets that are untouched
def UntouchedFacets(inputSet, cheat = False, offset = True, setOutput = True):
    #constructs a set of faces that the input nodes connect to, and checks that against completeSetOfFaces
    solutionSet = set()
    if cheat:
        solutionSet = setOfFirst24Connections.copy() ##TODO make sure this is equivalent to adding [1,24]
        
    # add the facets connecting to the set in inputSet
    for i in inputSet:
        try:
            solutionSet.update(listOfNodeConnections[i - offset])
        except:
            print("data probably didn't import properly", i) 
            
    #len comparison is obviously much faster, but smart algo needs the set of facets that are untouched
    #so this is used instead of copying code
    if setOutput:
        return completeSetOfFaces.difference(solutionSet) 
    return len(completeSetOfFaces) - len(solutionSet)
    


def FacetConnections(inputSet, exclusive = True, offset = True):
    output = set()
    for facet in inputSet:
        output.update(listOfFacetConnections[facet - offset])
    #if exclusive:
    #    output = output.difference(inputSet)
    return output

In [ ]:
# Function call / Testing area

In [15]:
# Probably faster, but I think it wont hit as many solutions
chainTestingSet = set(list(range(1,25)))
chainTestingSet.update([55, 56, 43, 42, 95, 106, 77, 81, 75, 66, 69, 87, 65, 109, 34, 37, 97, 51, 32, 114, 119, 107, 96, 26, 120, 118])
print(len(Backtrack(chainTestingSet.difference([120]), 1)))
print(Backtrack(chainTestingSet.difference([120]),1 ))

chainTestingOutput = chainBacktrack(chainTestingSet, 2, 1)
#print(chainTestingOutput)
print("length of starting set", len(chainTestingSet))
print("length of output sets", len(chainTestingOutput[0]))

#testA = set([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 32, 34, 37, 42, 43, 51, 55, 56, 65, 66, 69, 75, 77, 81, 87, 95, 96, 97, 106, 107, 109, 114, 118, 119])
#print(Backtrack(testA, 1))

1
[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 32, 34, 37, 42, 43, 51, 55, 56, 65, 66, 69, 75, 77, 81, 87, 95, 96, 97, 106, 107, 109, 114, 119}]
hey [{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 32, 34, 37, 42, 43, 51, 55, 56, 65, 66, 69, 75, 77, 81, 87, 95, 96, 97, 106, 107, 109, 114, 119}, {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 32, 34, 37, 42, 43, 51, 55, 56, 65, 66, 69, 75, 77, 81, 87, 95, 96, 97, 106, 107, 109, 114, 119}]
length of starting set 50
length of output sets 48


In [16]:
fullSetOfNodes = set(range(1, 121))
tempbadset = set(list(range(1,25)))
tempbadset.update([64, 60, 108, 38, 101, 59, 115, 41, 73, 27, 103, 72, 82, 52, 30, 78, 120, 98, 89, 92, 91, 70, 35, 113, 36, 40, 47])
print("Initial set", tempbadset, "\nLength", len(tempbadset))

nodesToRemove = 1
nodesToReplace = 1
tic = time.time()
dumbResult = Backtrack(tempbadset, nodesToRemove, numReplacements = nodesToReplace, smart = False)
#print(dumbResult)
toc = time.time()
print("DumbBacktrack Runtime: ", toc-tic)

tic = time.time()
smartResult = Backtrack(tempbadset, nodesToRemove, numReplacements = nodesToReplace, smart =  True)
#print(smartResult)
toc = time.time()
print("SmartBacktrack Runtime:", toc-tic)

print("DumbBacktrack Solutions:", len(dumbResult))
#print(dumbResult)
print("SmartBacktrack Solutions:", len(smartResult))
#print(smartResult)

#if they're different lengths, print out how many duplicates there are
if len(dumbResult) != len(smartResult):
    counter = 1
    for i in dumbResult:
        flag = True
        for j in smartResult:
            if i.difference(j) == set():
                print("Duplicate:", counter)
                #print i
                counter += 1
                flag = False
        if flag:
            print(i)
        
print("Solution Length:", len(dumbResult[0]))

Initial set {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 27, 30, 35, 36, 38, 40, 41, 47, 52, 59, 60, 64, 70, 72, 73, 78, 82, 89, 91, 92, 98, 101, 103, 108, 113, 115, 120} 
Length 51
DumbBacktrack Runtime:  0.1759648323059082
SmartBacktrack Runtime: 0.08504223823547363
DumbBacktrack Solutions: 80
SmartBacktrack Solutions: 80
Solution Length: 51


In [8]:
#just testing speed of different comparisons
print(len(dumbResult[0]))

completeSetOfFaces2 = completeSetOfFaces.copy()
tic = time.time()
for i in range(10000):
    temptest = completeSetOfFaces.difference(completeSetOfFaces2)
toc = time.time()
print(toc-tic)

tic = time.time()
for i in range(10000):
    temptest = len(completeSetOfFaces) - len(completeSetOfFaces2)
toc = time.time()
print(toc-tic)

51
0.13599133491516113
0.0030007362365722656


Filepath to import
sixhundredcell_3_corrected.csv
File Loaded


In [6]:
#print(numNodes)
#print(tempSet)
#print(len(listOfFacetConnections))
#print(listOfFacetConnections)